# Part 4: Accessing and Analyzing PHYSLITE Data

In [ ]:
# Import basic libraries
import copy # copy variables
import os   # manage paths

import uproot   # use of root files
import awkward as ak    # nested, variable sized data
import vector   # lorentz vectors
vector.register_awkward()
import matplotlib.pyplot as plt # plotting
import matplotlib as mpl # plotting
import tqdm # progress bars
import atlasopenmagic as atom # ATLAS Open Data package

In [ ]:
atom.get_current_release()

In [ ]:
urls_sample = atom.get_urls(410470, protocol='https')
urls_sample[:5]

In [ ]:
# mc20_13TeV.410470.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.deriv.DAOD_PHYSLITE.e6337_s3681_r13167_p5855
filename = urls_sample[0]
filename

## Read PHYSLITE with uproot

In [ ]:
print('TTree objects inside the ROOT file:')
for ii in uproot.open(filename).keys():
    print('-',ii)

In [ ]:
tree = uproot.open({filename: "CollectionTree"})

### List branches

In [ ]:
# Display only the first 10 branches
first_10_branches = list(tree.keys())[:10]
tree.show(filter_name=first_10_branches, name_width=50, typename_width=50, interpretation_width=50)

### Load branches into awkward and numpy arrays

In [ ]:
el_pt = tree["AnalysisElectronsAuxDyn.pt"].array()

In [ ]:
el_pt

In [ ]:
el_pt_np = ak.flatten(el_pt).to_numpy()
print(f'Total number of electrons in 150,000 events: {len(el_pt_np):,}')
el_pt_np

In [ ]:
plt.hist(el_pt_np, bins=100)
# plt.yscale('log')
plt.title('$p_T$ distribution of all $e$')
plt.xlabel('$p_T$ [MeV]')
plt.ylabel('Number of electrons')
plt.show()

**Quizlet:** Plot the pseudorapidity (eta) of the jets and the electrons on the same figure. Do you know why the two distributions span over different range?

**Answer:**

In [ ]:
# Your code goes here

## Simple analysis

### Collect branches into records

In [ ]:
electrons = ak.zip(
    {
        "pt": tree["AnalysisElectronsAuxDyn.pt"].array(),
        "eta": tree["AnalysisElectronsAuxDyn.eta"].array(),
        "phi": tree["AnalysisElectronsAuxDyn.phi"].array(),
    }
)

muons = ak.zip(
     {
        "pt": tree["AnalysisMuonsAuxDyn.pt"].array(),
        "eta": tree["AnalysisMuonsAuxDyn.eta"].array(),
        "phi": tree["AnalysisMuonsAuxDyn.phi"].array(),
    }
)

jets = ak.zip(
     {
        "pt": tree["AnalysisJetsAuxDyn.pt"].array(),
        "eta": tree["AnalysisJetsAuxDyn.eta"].array(),
        "phi": tree["AnalysisJetsAuxDyn.phi"].array(),
        "mass": tree["AnalysisJetsAuxDyn.m"].array(),
    }
)

In [ ]:
electrons

In [ ]:
electrons.pt

In [ ]:
electrons["pt"]

In [ ]:
btag_prob = tree["BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pb"].array()

**Quizlet:** Check that the number of btagging values equals to the number of jets in the file.

**Answer:**

In [ ]:
# Your code goes here

In [ ]:
jets["btag_prob"] = btag_prob
jets

In [ ]:
events = ak.zip({"Electrons": electrons, "Muons": muons, "Jets": jets}, depth_limit=1)
init_number_of_events = len(events)
events

### Event and object selection

#### Basic object selections

In [ ]:
GeV = 1000.
mask = electrons.pt > 30 * GeV
mask

In [ ]:
electrons[mask]

In [ ]:
def selected_electrons(el):
    return el[(el.pt > 30 * GeV) & (abs(el.eta) < 2.47)]

def selected_muons(mu):
    return mu[(mu.pt > 30 * GeV) & (abs(mu.eta) < 2.47)]

def selected_jets(j):
    return j[(j.pt > 30 * GeV) & (abs(j.eta) < 2.47)]

In [ ]:
events["Electrons"] = selected_electrons(electrons)
events["Muons"] = selected_muons(muons)
events["Jets"] = selected_jets(jets)

In [ ]:
print(f'Number of electrons before selection: {ak.count(electrons.pt):,}')
print(f'Number of electrons after selection: {ak.count(events.Electrons.pt):,}')

#### Lorentz vectors

In [ ]:
events["Electrons"] = vector.awk(events.Electrons)
events["Muons"] = vector.awk(events.Muons)
events["Jets"] = vector.awk(events.Jets)

In [ ]:
events.Electrons

In [ ]:
events.Electrons.px

#### Overlap removal

In [ ]:
jj, ee = ak.unzip( ak.cartesian([events.Jets, events.Electrons], nested=True) )
plt.hist(ak.flatten(jj.deltaR(ee), axis=None).to_numpy(), bins=100)
plt.xlabel(r"$\Delta R(j, e)$ (for all jet–electron pairs)")
plt.ylabel("Count of Jet-Electron Pairs")
plt.title(r"Distribution of $\Delta R$ Between Jets and Electrons")
plt.show()

In [ ]:
def no_overlap(obj1, obj2, deltaR=0.4):
    obj1, obj2 = ak.unzip(ak.cartesian([obj1, obj2], nested=True))
    return ak.all(obj1.deltaR(obj2) > deltaR, axis=-1)

In [ ]:
no_overlap(events.Jets, events.Electrons) # mask for each jet if it has no overlap with any electron

In [ ]:
events["Jets"] = events.Jets[no_overlap(events.Jets, events.Electrons)]

#### Apply event selection

In [ ]:
events["Jets", "is_bjet"] = events.Jets.btag_prob > 0.85

events = events[
    (ak.num(events.Jets) >= 4) # at least 4 jets
    & ((ak.num(events.Electrons) + ak.num(events.Muons)) == 1) # exactly one lepton
    & (ak.num(events.Jets[events.Jets.is_bjet]) >= 2) # at least two btagged jets with prob > 0.85
]

In [ ]:
print(f'Initially we had {init_number_of_events:,} events. After the event selection we have {len(events):,} events')

### Top quark reconstruction

In [ ]:
def mjjj(jets):
    candidates = ak.combinations(jets, 3)
    j1, j2, j3 = ak.unzip(candidates)
    candidates["p4"] = j1 + j2 + j3
    has_b = (j1.is_bjet + j2.is_bjet + j3.is_bjet) > 0
    candidates = candidates[has_b]
    candidates = candidates[ak.argmax(candidates.p4.pt, axis=1, keepdims=True)]
    return candidates.p4.mass

In [ ]:
plt.hist(ak.flatten(mjjj(events.Jets) / GeV, axis=None), bins=100)
plt.xlabel("Reconstructed Top Quark Mass (GeV)")
plt.ylabel("Number of Events")
plt.title("Distribution of Reconstructed Top Quark Mass")
plt.axvline(172.76, color='r', linestyle='dashed', linewidth=2, label='Expected Top Quark Mass')
plt.legend()
plt.show()